In [10]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

# age, lulc, fire = desired_lulc()

# remapped_image = lulc.bandNames().map(lambda band_name: remap_band(band_name, lulc))
# remapped_image = ee.ImageCollection(remapped_image).toBands()
# desired_mask = remapped_image.reduce("sum").eq(lulc.bandNames().size().getInfo())

# age = age.updateMask(desired_mask).rename("age")
# lulc = lulc.updateMask(desired_mask)
# fire = fire.updateMask(desired_mask)

history_interval = 0
use_hard_cutoff = True

aggregate_LU_types = False

name = ""

if aggregate_LU_types:
    name_LU = f"{name}_aggregated"
else:
    name_LU = f"{name}_non_aggregated"

if use_hard_cutoff:
    name_LU += "_hard_cutoff"

if history_interval > 0:
    name += f"_{history_interval}yr"
else:
    name += "_all"


fire_eu = True

## Land Use Land Cover

### Grouping land use types (if aggregation is desired)

- annual_crops = 39, 20, 40, 62, 41
- perennial_crops = 46, 47, 35, 48
- pasture = 15
- mosaic = 21
- forest plantation = 9


In [257]:

# Listing land use types that are NOT forest
if (aggregate_LU_types):
    
    LU_index = [10,20,30,40,50]

    # A flat list of pixel values to replace.
    from_list = [15, 46, 47, 35, 48, 39, 20, 40, 62, 41, 21, 9, 3]

    # A corresponding list of new values.
    to_list = [10, 20, 20, 20, 20, 30, 30, 30, 30, 30, 40, 50, 3]

    # for each forest age, mask the lulc of the year immediately preceding abandonment
    lulc_aggregated = ee.Image()
    for band in lulc.bandNames().getInfo():
        lu_yr = lulc.select(band)
        remapped_band = lu_yr.remap(from_list, to_list).rename(band)
        lulc_aggregated = lulc_aggregated.addBands(remapped_band)

    lulc = lulc_aggregated.slice(1).byte()
else:
    LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21]



### Restricting history to n years before abandonment

Since we only have land use history dating back to 1985, the data is censored. In order to restrict history to exclude pixels with possible long histories of anthropogenic use pre-1985 which could not be included in the analysis:

- Make a mask with regrowing pixels that were forested in 1985 (keep only the patches that were forested in 1985, then subsequently deforested, and regrew after that).

- Making a mask including only pixels that have a specified number of bands (hist_interval) between the first and the last occurrence of an anthropogenic pixel (desired extent of land use history to include)

In [258]:

# for each year, make a mask for all anthropogenic pixels, and multiply by the year to have the
# pixel value be the year of observation of that land use type.
non_forest_years = ee.Image()
for yr in range(1985, 2020):
    year = f"{yr}"
    lu_yr = lulc.select(year).gt(3).multiply(yr)
    non_forest_years = non_forest_years.addBands(lu_yr)

# find the minimum and maximum non-zero value per pixel across all bands
# (first and last year of anthropogenic land use)
first_anthro = non_forest_years.selfMask().reduce(ee.Reducer.min())
last_anthro = non_forest_years.reduce(ee.Reducer.max())
# get total of continuous years from the first and last observation of anthropogenic land use
anthro_gaps = last_anthro.subtract(first_anthro).add(1)



In [4]:

# Establishing a specific range for land use history for analysis
if history_interval > 0:
    first_year_considered = 1984 + history_interval
    age = age.updateMask(age.lte(2020 - first_year_considered))
    years_regrowth_considered = range(first_year_considered, 2020)

    if use_hard_cutoff:
        # Calculate the cutoff year for each pixel
        cutoff_year = ee.Image.constant(2021).subtract(age).subtract(ee.Number(history_interval))
        years = ee.Image.constant(list(range(1985, 2021))).rename(lulc.bandNames())
        mask = years.gte(cutoff_year)
        lulc = lulc.updateMask(mask)
    else:
        # select pixels that were forested in 1985
        # and that have at most history_interval years of anthropogenic land use
        lulc = lulc.updateMask(lulc.select("1985").eq(3)).selfMask() \
        .updateMask(age).updateMask(anthro_gaps.lte(history_interval))

else:
    years_regrowth_considered = config.range_1985_2019


### Fallow period length

Finds the total number of years that land was left fallow (classified as forest before regrowth)

In [260]:
nat_cover = lulc.eq(3)
nat_cover = nat_cover.reduce(ee.Reducer.sum())

if history_interval > 0 and use_hard_cutoff:
    # fallow = nat_cover.subtract(age).rename("fallow").byte()

    non_forest_years = ee.Image()
    for yr in range(1985, 2020):
        year = f"{yr}"
        lu_yr = lulc.select(year).gt(3).multiply(yr)
        non_forest_years = non_forest_years.addBands(lu_yr)

    # find the minimum and maximum non-zero value per pixel across all bands
    # (first and last year of anthropogenic land use)
    first_anthro = non_forest_years.selfMask().reduce(ee.Reducer.min())
    forest_before_first_anthro = first_anthro.subtract(cutoff_year)
    fallow = nat_cover.subtract(forest_before_first_anthro).subtract(age).rename("fallow").byte()
else:
    forest_before_first_anthro = first_anthro.subtract(1985)
    fallow = nat_cover.subtract(forest_before_first_anthro).subtract(age).rename("fallow").byte()

### Total sum of years under each desired land use type

For each desired land use type, the total occurrences per pixel are summed

In [261]:
LU_sum = ee.Image()
for val in LU_index:
    lulc_val = lulc.eq(val)
    num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f"lulc_sum_{val}")
    LU_sum = LU_sum.addBands(num_cells)

# removes the first (empty) band
LU_sum = LU_sum.slice(1).byte()

### Last Land Use Type

Finds what was the last land use type observed before abandonment.

In [262]:
# for each forest age, mask the lulc of the year immediately preceding abandonment
last_LU = ee.Image()
for yr in config.range_1986_2019:
    year = f"{yr}"
    lu_yr = lulc.select(year)
    age_mask = age.eq(config.last_year - yr)
    last_LU = last_LU.addBands(lu_yr.updateMask(age_mask))

# remove first empty band
# merge all images by adding them up
last_LU = last_LU.slice(1).reduce(ee.Reducer.sum()).rename("last_LU").byte()

### Export land_use

In [263]:
land_use = LU_sum.addBands([last_LU, fallow])

export_image(land_use, f"land_use{name_LU}", scale = 30, folder = "mapbiomas")


## Fire

### Total number of fires per pixel

Each burned pixel contains the value of the land use type that burned (such as the desired_values vector specified in the land use section) - unburned pixels are zero.

There are instances of fires:
- after the first observation of anthropogenic land use
- before the first observation of anthropogenic land use (forest fire)
- after the last observation of anthropogenic land use (burn in regrowing forest)

*Note that fire has different transform than lulc, and the projections will be adjusted when exporting.*

### How many years ago was the last fire?

Make one image with 34 bands, each with the number of years from the instance of burn to 2020.
Each pixel is 0 if unburned, and "time since fire" in number of years if it's burned.

In [7]:
if fire_eu:
    age = ee.Image(f"{data_folder}/eu/secondary").select("age")
    name = "_eu"
    folder = "eu"
    fire = (ee.Image("projects/mapbiomas-public/assets/brazil/fire/collection3/mapbiomas_fire_collection3_annual_burned_coverage_v1")
        .select([f"burned_coverage_{year}" for year in config.range_1985_2020])
        .byte()
        .rename([str(year) for year in config.range_1985_2020])
        .updateMask(age))
else:
    folder = "mapbiomas"

# Making a fire mask with pixels of value 1 for burned pixels and 0 otherwise
fire = fire.gt(0)

"""
Establishing a specific range for land use history for analysis, 
the ages of forests regrowing are also restricted.
For example, if we have a fixed 10 years of history:
1985 ---------- 10 years ----------- 1995 ----------- 25 years -----------2020
the regrowing forests have started regrowing in 1995 at the very earliest.

Then, the years of regrowth considered are:
"""

# Initialize images
num_fires_before_regrowth = ee.Image()
num_fires_after_regrowth = ee.Image()

# Calculate burn information and add yearly bands to the corresponding images
for yr in years_regrowth_considered:
    yrs_ago = 2020 - yr
    age_mask = age.eq(yrs_ago).selfMask()
    
    if yr < 2019:
        # Add bands for after regrowth
        bands_after = fire.select([str(year) for year in range(yr + 1, 2020)]).updateMask(age_mask)
        num_fires_after_regrowth = num_fires_after_regrowth.addBands(
            bands_after.reduce(ee.Reducer.sum()).byte().rename(f"{yrs_ago}")
        )
    
    # Add bands for before regrowth
    bands_before = fire.select([str(year) for year in range(1985, yr+1)]).updateMask(age_mask)
    num_fires_before_regrowth = num_fires_before_regrowth.addBands(
        bands_before.reduce(ee.Reducer.sum()).byte().rename(f"{yrs_ago}")
    )

# Remove the first band (assuming it's an unwanted initial band)
num_fires_before_regrowth = num_fires_before_regrowth.slice(1).reduce(ee.Reducer.sum()).selfMask() \
    .rename("num_fires_before_regrowth")
num_fires_after_regrowth = num_fires_after_regrowth.slice(1).reduce(ee.Reducer.sum()).selfMask() \
    .rename("num_fires_after_regrowth")

# Combine the before and after regrowth fire counts
num_fires = num_fires_before_regrowth.addBands(num_fires_after_regrowth).byte()

# Unmask to fill any missing values with 0
num_fires = num_fires.unmask(0)

export_image(num_fires, f"num_fires{name}", scale = 30, folder = folder)